In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, scale!, evaluate
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1260


### Greedy fit

In [160]:
Nx = 2
Ne = 8
m = 5
ens = EnsembleState(Nx, Ne)

ens.S .=  [0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]';
X = ens.S

coeff = [0.6285037650645056;
          -0.4744029092496623;
           1.1405280011620331;
          -0.7217760771894809;
           0.11855056306742319];
# Initialize map with zero coefficients
Hk_old = HermiteMapk(m, Nx, [0 0; 0 1; 0 2; 1 0; 2 0], coeff; α = 1e-6);

S = Storage(Hk_old.I.f, X)

reduced_margin = getreducedmargin(getidx(Hk_old))
@show reduced_margin
@time idx_new, reduced_margin = update_component(Hk_old, X, reduced_margin, S)

@show idx_new
@show reduced_margin

# idx_new = vcat(getidx(Hk_old), reduced_margin)
# Hk_new = HermiteMapk(m, Nx, idx_new, vcat(getcoeff(Hk_old), zeros(size(reduced_margin,1))); α = 1.0);
# # coeff_new, coeff_idx_added, idx_added = update_coeffs(Hk_old, Hk_new)

reduced_margin = [0 3; 1 1; 3 0]
  0.000235 seconds (1.53 k allocations: 90.188 KiB)
idx_new = [0 0; 0 1; 0 2; 1 0; 2 0; 0 3]
reduced_margin = [1 1; 3 0; 0 4]


In [171]:
Nx = 2
Ne = 8
m = 5
ens = EnsembleState(Nx, Ne)

ens.S .=  [0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]';
X = ens.S
# Initialize map with zero coefficients
Hk_old = HermiteMapk(m, Nx; α = 1e-6);

S = Storage(Hk_old.I.f, X)

# setcoeff!(Hk_old, [1.5])
reduced_margin0 = getreducedmargin(getidx(Hk_old))
@show reduced_margin0

idx_new, reduced_margin = update_component(Hk_old, X, reduced_margin0, S)

reduced_margin0 = [0 1; 1 0]


([0 0; 0 1], [1 0; 0 2])

In [235]:
# @testset "Test update_component I" begin
    Nx = 2
    Ne = 8
    m = 5
    ens = EnsembleState(Nx, Ne)

    ens.S .=  [0.267333   1.43021;
              0.364979   0.607224;
             -1.23693    0.249277;
             -2.0526     0.915629;
             -0.182465   0.415874;
              0.412907   1.01672;
              1.41332   -0.918205;
              0.766647  -1.00445]';
    X = ens.S

    idx = [0 0; 0 1; 1 0; 0 2; 2 0; 1 1]

    coeff = [ -0.9905841755746164;
              0.6771992097558741;
             -2.243695806805015;
             -0.34591297359447354;
             -1.420159186008486;
             -0.5361337327704369]
    # Initialize map with zero coefficients
    Hk_old = HermiteMapk(m, Nx, idx, coeff; α = 1e-6);

    S = Storage(Hk_old.I.f, X)

    reduced_margin0 = getreducedmargin(getidx(Hk_old))
    idx_old0 = getidx(Hk_old)
    idx_new0 = vcat(idx_old0, reduced_margin0)

10×2 Array{Int64,2}:
 0  0
 0  1
 1  0
 0  2
 2  0
 1  1
 0  3
 1  2
 2  1
 3  0

In [236]:
f_new = ExpandedFunction(Hk_old.I.f.f.B, idx_new0, vcat(getcoeff(Hk_old), zeros(size(reduced_margin0,1))))
Hk_new = HermiteMapk(f_new; α = 1e-6)

HermiteMapk{5,10,2}(IntegratedFunction{5,10,2}(Rectifier("softplus"), ParametricFunction{5,10,2}(ExpandedFunction{5,10,2}(MultiBasis{5,2}(Basis of 5 functions:
ProPolyHermite{0}(ImmutablePolynomial(1.0), false)
ProHermite{0}(ProPolyHermite{0}(ImmutablePolynomial(0.6316187777460647), true), true)
ProHermite{1}(ProPolyHermite{1}(ImmutablePolynomial(0.6316187777460647*x), true), true)
ProHermite{2}(ProPolyHermite{2}(ImmutablePolynomial(-0.44662192086900115 + 0.44662192086900115*x^2), true), true)
ProHermite{3}(ProPolyHermite{3}(ImmutablePolynomial(-0.7735718587191167*x + 0.25785728623970555*x^3), true), true)
), [0 0; 0 1; … ; 2 1; 3 0], [-0.9905841755746164, 0.6771992097558741, -2.243695806805015, -0.34591297359447354, -1.420159186008486, -0.5361337327704369, 0.0, 0.0, 0.0, 0.0]))), 1.0e-6)

In [237]:
idx_new, reduced_margin = update_component(Hk_old, X, reduced_margin0, S)


([0 0; 0 1; … ; 1 1; 0 3], [1 2; 2 1; 3 0; 0 4])

In [238]:
reduced_margin0

4×2 Array{Int64,2}:
 0  3
 1  2
 2  1
 3  0

In [239]:
idx_new0

10×2 Array{Int64,2}:
 0  0
 0  1
 1  0
 0  2
 2  0
 1  1
 0  3
 1  2
 2  1
 3  0

In [240]:
idx_new

7×2 Array{Int64,2}:
 0  0
 0  1
 1  0
 0  2
 2  0
 1  1
 0  3

In [241]:
    # Define updated map

    
    dJ = zeros(10)
    S = update_storage(S, X, reduced_margin0)

    negative_log_likelihood!(nothing, dJ, getcoeff(Hk_new), S, Hk_new, X)
# # # # end

In [247]:
dJ

10-element Array{Float64,1}:
 -1.6651836742291564
 -0.9194410618771688
 -0.9009267357141437
 -0.11952335389279932
 -0.4727973085063471
 -0.49686342659213534
  0.3388316230967308
 -0.0835027246673459
 -0.2811305337405587
  0.27158719056266994

In [244]:
findmax(abs.(dJ[7:10]))

(0.3388316230967308, 1)

In [245]:
idx_new

7×2 Array{Int64,2}:
 0  0
 0  1
 1  0
 0  2
 2  0
 1  1
 0  3

In [246]:
reduced_margin0[1,:]

2-element Array{Int64,1}:
 0
 3

In [187]:
    @test idx_new[2,:] == reduced_margin0[1,:]


Test Passed

In [243]:
reduced_margin0 

4×2 Array{Int64,2}:
 0  3
 1  2
 2  1
 3  0

In [189]:
    @test reduced_margin == updatereducedmargin(getidx(Hk_old), reduced_margin0, 1)


Test Failed at In[189]:1
  Expression: reduced_margin == updatereducedmargin(getidx(Hk_old), reduced_margin0, 1)
   Evaluated: [1 0; 0 2] == ([0 0; 0 1], [1 0; 0 2])


Test.FallbackTestSetException: There was an error during testing

In [191]:
    Nx = 2
    Ne = 8
    m = 5
    ens = EnsembleState(Nx, Ne)

    ens.S .=  [0.267333   1.43021;
              0.364979   0.607224;
             -1.23693    0.249277;
             -2.0526     0.915629;
             -0.182465   0.415874;
              0.412907   1.01672;
              1.41332   -0.918205;
              0.766647  -1.00445]';
    X = ens.S
    # Initialize map with zero coefficients
    Hk_old = HermiteMapk(m, Nx; α = 1e-6);

    S = Storage(Hk_old.I.f, X)

    # setcoeff!(Hk_old, [1.5])
    reduced_margin0 = getreducedmargin(getidx(Hk_old))
    idx_old0 = getidx(Hk)
    idx_new0 = vcat(idx_old, reduced_margin0)

    # Define updated map
    f_new = ExpandedFunction(Hk_old.I.f.f.B, idx_new0, vcat(getcoeff(Hk), zeros(size(reduced_margin0,1))))
    Hk_new = HermiteMapk(f_new; α = 1e-6)
    idx_new, reduced_margin = update_component(Hk_old, X, reduced_margin0, S)

    dJ = zeros(3)
    S = update_storage(S, X, reduced_margin0)
    negative_log_likelihood!(nothing, dJ, getcoeff(Hk_new), S, Hk_new, X)

    @test findmax(abs.(dJ[2:3]))[2] == 2-1

    @test idx_new[2,:] == reduced_margin0[1,:]

    @test reduced_margin == updatereducedmargin(getidx(Hk_old), reduced_margin0, 1)[2]
    @test idx_new == updatereducedmargin(getidx(Hk_old), reduced_margin0, 1)[1]

Test Passed